In [273]:
import sys
sys.path.append('./tismir-beat-2024/')
import os

import numpy as np
import pandas as pd

In [274]:
ANNOTATION_DIR = './data/IEMP-condombe/Annotations'
AUDIO_DIR = './data/IEMP-condombe/Media'

In [275]:
dfs = {}

for root, dirs, files in os.walk(ANNOTATION_DIR):
    for file in files:
        if file.endswith('.csv') and root.split('/')[-1].startswith('UC'):
            if dfs.get(root.split('/')[-1]) is None:
                dfs[root.split('/')[-1]] = []

dfs_madera = {}
for root, dirs, files in os.walk(os.path.join(ANNOTATION_DIR, 'madera')):
    for file in files:
        if file.endswith('.csv'):
            df = pd.read_csv(os.path.join(root, file), header=None)
            df.columns = ['Cycle', 'label']
            dfs_madera[file.split('_')[1]+file.split('_')[2]] = df
                


In [276]:
import math

for df in dfs:
    dfs[df] = pd.read_csv(os.path.join(ANNOTATION_DIR, df, f'{df}_Metre.csv'))
    dfs[df].columns = ['Beat', 'Metre']
    dfs[df]['Bar'] = dfs[df]['Beat'].apply(lambda x: math.floor(x))
    for df_madera in dfs_madera:
        if df_madera.__contains__(df.split('_')[1]):
            dfs[df] = pd.merge(dfs[df], dfs_madera[df_madera], left_on='Bar', right_on='Cycle', how='left')

In [277]:
df_madera

'321repique1'

In [278]:
dfs['UC_211']

,Beat,Metre,Bar,Cycle,label
0,1.1,3.410476,1,1.0,mix
1,1.2,4.031143,1,1.0,mix
2,1.3,4.640476,1,1.0,mix
3,1.4,5.235143,1,1.0,mix
4,2.1,5.840476,2,2.0,madera
...,...,...,...,...,...
332,84.1,205.194226,84,84.0,repique
333,84.2,205.782976,84,84.0,repique
334,84.3,206.390476,84,84.0,repique
335,84.4,206.995143,84,84.0,repique


In [279]:
for df in dfs:
    dfs[df].rename(columns={'Metre': 'start'}, inplace=True)
    dfs[df]['end'] = dfs[df]['start'].shift(-1)
    
    if 'label_x' in dfs[df].columns:
        dfs[df]['contains_madera'] = dfs[df]['label_x'].apply(lambda x: x == 'madera' or x == 'mix') | dfs[df]['label_y'].apply(lambda x: x == 'madera' or x == 'mix')
        dfs[df] = dfs[df].drop(columns=['Bar', 'label_x', 'label_y', 'Cycle_x', 'Beat', 'Cycle_y'])
    else:
        dfs[df]['contains_madera'] = dfs[df]['label'].apply(lambda x: x == 'madera' or x == 'mix')
        dfs[df] = dfs[df].drop(columns=['Bar', 'label', 'Cycle'])


In [280]:
from IPython.display import HTML

display(dfs['UC_211'])
html = dfs['UC_211'].to_html(classes='table table-striped', index=True)
HTML(f'<div style="height: 300px; overflow: auto;">{html}</div>')

,Beat,start,end,contains_madera
0,1.1,3.410476,4.031143,True
1,1.2,4.031143,4.640476,True
2,1.3,4.640476,5.235143,True
3,1.4,5.235143,5.840476,True
4,2.1,5.840476,6.453810,True
...,...,...,...,...
332,84.1,205.194226,205.782976,False
333,84.2,205.782976,206.390476,False
334,84.3,206.390476,206.995143,False
335,84.4,206.995143,207.613205,False


,Beat,start,end,contains_madera
0,1.1,3.410476,4.031143,True
1,1.2,4.031143,4.640476,True
2,1.3,4.640476,5.235143,True
3,1.4,5.235143,5.840476,True
4,2.1,5.840476,6.453810,True
5,2.2,6.453810,7.048476,True
6,2.3,7.048476,7.632476,True
7,2.4,7.632476,8.227143,True
8,3.1,8.227143,8.825476,True
9,3.2,8.825476,9.429810,True


In [281]:
# remove rows that don't contain madera
for df in dfs:
    dfs[df] = dfs[df][dfs[df]['contains_madera']]
    dfs[df] = dfs[df].drop(columns=['contains_madera'])



In [282]:
segments = pd.DataFrame(columns=['start', 'end', 'file'])
for df in dfs:
    temp_df = dfs[df][['start', 'end']].copy()
    temp_df['file'] = df
    segments = pd.concat([segments, temp_df], ignore_index=True)

segments

/tmp/ipykernel_10662/2465773886.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  segments = pd.concat([segments, temp_df], ignore_index=True)


,start,end,file
0,0.141815,0.702857,UC_221
1,0.702857,1.234857,UC_221
2,1.234857,1.777524,UC_221
3,1.777524,2.273690,UC_221
4,2.273690,2.810857,UC_221
...,...,...,...
1595,171.430077,171.894077,UC_312
1596,171.894077,172.368744,UC_312
1597,172.368744,172.839410,UC_312
1598,172.839410,173.310452,UC_312


In [283]:
merged_segments = []
current = None

for idx, row in segments.iterrows():
    if current is None:
        current = row
    else:
        # Check if segments are continuous
        if current['end'] == row['start']:
            # Merge segments
            current['end'] = row['end']
        else:
            # Store current segment and start new one
            merged_segments.append(current)
            current = row

# Add last segment
if current is not None:
    merged_segments.append(current)

# Create new DataFrame with merged segments
merged_df = pd.DataFrame(merged_segments)

In [284]:
print(f'Merged {len(segments)} segments into {len(merged_segments)} segments')

html = merged_df.to_html(classes='table table-striped', index=True)
HTML(f'<div style="height: 300px; overflow: auto;">{html}</div>')

Merged 1600 segments into 66 segments


,start,end,file
0,0.141815,17.749940,UC_221
36,28.887440,30.698857,UC_221
40,70.979315,74.616190,UC_221
48,97.859315,101.437524,UC_221
56,121.531190,125.198857,UC_221
64,134.412149,141.799482,UC_221
80,156.609524,158.517524,UC_221
84,167.936191,169.845524,UC_221
88,0.574884,10.226446,UC_231
104,12.615196,14.983321,UC_231


In [285]:
for root, dirs, files in os.walk(AUDIO_DIR):
    for file in files:
        if file.split('_')[-1] == 'Stereo.wav':
            id = file.split('_')[0]+'_'+file.split('_')[1]
            print(id)
            df = merged_df[merged_df['file'] == id]

            for idx, row in df.iterrows():
                start = row['start']
                end = row['end']
                print(f'ffmpeg -i "{os.path.join(root, file)}" -ss {start} -to {end} "{os.path.join(root, id)}_madera_{idx}.wav"')
                os.system(f'ffmpeg -i "{os.path.join(root, file)}" -ss {start} -to {end} "{os.path.join(root, id)}_madera_{idx}.wav"')

print('Done')

ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

UC_221
ffmpeg -i "./data/IEMP-condombe/Media/UC_221/UC_221_Stereo.wav" -ss 0.141815476 -to 17.74994048 "./data/IEMP-condombe/Media/UC_221/UC_221_madera_0.wav"
ffmpeg -i "./data/IEMP-condombe/Media/UC_221/UC_221_Stereo.wav" -ss 28.88744048 -to 30.69885714 "./data/IEMP-condombe/Media/UC_221/UC_221_madera_36.wav"
ffmpeg -i "./data/IEMP-condombe/Media/UC_221/UC_221_Stereo.wav" -ss 70.97931548 -to 74.61619048 "./data/IEMP-condombe/Media/UC_221/UC_221_madera_40.wav"


ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

ffmpeg -i "./data/IEMP-condombe/Media/UC_221/UC_221_Stereo.wav" -ss 97.85931548 -to 101.4375238 "./data/IEMP-condombe/Media/UC_221/UC_221_madera_48.wav"
ffmpeg -i "./data/IEMP-condombe/Media/UC_221/UC_221_Stereo.wav" -ss 121.5311905 -to 125.1988571 "./data/IEMP-condombe/Media/UC_221/UC_221_madera_56.wav"


Output #0, wav, to './data/IEMP-condombe/Media/UC_221/UC_221_madera_56.wav':
  Metadata:
    ISFT            : Lavf60.16.100
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 44100 Hz, stereo, s16, 1411 kb/s
    Metadata:
      encoder         : Lavc60.31.102 pcm_s16le
[out#0/wav @ 0x5570f1ceb5c0] video:0kB audio:632kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.012056%
size=     632kB time=00:00:03.64 bitrate=1419.1kbits/s speed=42.1x    
ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfr

...

ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

ffmpeg -i "./data/IEMP-condombe/Media/UC_312/UC_312_Stereo.wav" -ss 75.7020771 -to 108.8127438 "./data/IEMP-condombe/Media/UC_312/UC_312_madera_1396.wav"
ffmpeg -i "./data/IEMP-condombe/Media/UC_312/UC_312_Stereo.wav" -ss 110.6558688 -to 162.5860771 "./data/IEMP-condombe/Media/UC_312/UC_312_madera_1468.wav"


Output #0, wav, to './data/IEMP-condombe/Media/UC_312/UC_312_madera_1396.wav':
  Metadata:
    ISFT            : Lavf60.16.100
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 44100 Hz, stereo, s16, 1411 kb/s
    Metadata:
      encoder         : Lavc60.31.102 pcm_s16le
[out#0/wav @ 0x5de9ed44a5c0] video:0kB audio:5704kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.001335%
size=    5704kB time=00:00:33.10 bitrate=1411.4kbits/s speed= 392x    
ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-li

ffmpeg -i "./data/IEMP-condombe/Media/UC_312/UC_312_Stereo.wav" -ss 164.4487438 -to 173.7752646 "./data/IEMP-condombe/Media/UC_312/UC_312_madera_1580.wav"
Done


ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena